In [ ]:
# BLOQUE 1: Importaciones y configuración inicial

import requests
from bs4 import BeautifulSoup
import sqlite3
import pandas as pd
import time
import re
from urllib.parse import urljoin, quote
import json

print("✅ Librerías importadas correctamente")
print("🕷️ Comenzando la infiltración en Books To Scrape...")


# BLOQUE 2: Funciones de Web Scraping 

def get_all_categories ():
    ''' OBTIENE TODAS LAS CATEGORIAS DEL SITIO'''
    url = "https://books.toscrape.com"
    respuesta = requests.get(url) # se descarga la web 
    soup = BeautifulSoup(respuesta.content, 'html.parser')  
    # convierte texto HTML ilegible, en un objeto inteligente que se pueda navegar 
    
    categories = []
    nav_list = soup.find('ul', class_='nav nav-list')
    if nav_list:
        category_links = nav_list.find_all ('a'[1:]) # Saltar "Books"
        for link in category_links:
            category_name = link.text.strip()
            category_url = urljoin(url, link['href'])
            categories.append({
                'name' : category_name,
                'url' : category_url
            })
            
    print (f"🎯 Encontradas {len(categories)} categorias")
    return categories 



            

In [ ]:

# Descargamos la Web 
url = "http://books.toscrape.com"
respuesta = requests.get(url) 

# BeautifulSoup procesa el HTML 
sopa = BeautifulSoup(respuesta.text, 'Html.parser') # convierte texto HTML ilegible, en un objeto inteligente que se pueda navegar 

# Exposicion: Se busca por ejemplo los titulos de los libros 
titulos = sopa.find_all('h3')
for titulo in titulos:
    print(titulo.get_text())
    

precios = sopa.find_all('p', class_ ='price_color') # sirve para encontrar todos los precios en la clase price_color 
for precio in precios:
    print(precio.get_text()) # el get funciona para conseguir todo el texto que yo le estoy pidiendo   
    

In [ ]:

# BLOQUE 6: Diseño UML y Estructura de Base de Datos


print("""
🏗️ DIAGRAMA UML - Estructura de la Base de Datos

┌─────────────────┐    ┌──────────────────┐    ┌─────────────────┐
│     AUTORES     │    │  LIBROS_AUTORES  │    │     LIBROS      │
├─────────────────┤    │   (Tabla Pivot)  │    ├─────────────────┤
│ id (PK)         │◄───┤ autor_id (FK)    │───►│ id (PK)         │
│ nombre          │    │ libro_id (FK)    │    │ titulo          │
│                 │    └──────────────────┘    │ precio          │
└─────────────────┘                            │ rating           │
                                            │ categoria          │
┌─────────────────┐                           │ en_stock         │
│   CATEGORIAS    │                           │ fecha_publicacion│
├─────────────────┤                           │ descripcion      │
│ id (PK)         │◄──────────────────────────│ categoria_id (FK)│
│ nombre          │                           │ url              │
│                 │                           └─────────────────┘
└─────────────────┘

Relaciones:
- AUTORES ↔ LIBROS: Muchos a Muchos (un autor puede escribir varios libros, 
  un libro puede tener varios autores)
- CATEGORIAS ↔ LIBROS: Uno a Muchos (una categoría tiene muchos libros)
""")

In [ ]:
# Creacion de base de datos ( DDL) 